In [1]:

import os
import json
import glob
import copy
import numpy as np
import pylab as pl
import seaborn as sns
import pandas as pd


from scipy import stats as spstats

from pipeline.python.classifications import experiment_classes as util
from pipeline.python.utils import natural_keys, label_figure

/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/repos/2p-pipeline/pipeline/python/classifications/experiment_classes.py:18: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/envs/pipeline/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/site-packages/traitlets/config/applica

/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/repos/2p-pipeline/pipeline/python/retinotopy/target_visual_field.py:19: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/net/coxfs01/srv/export/coxfs01/share_root/2p-pipeline/envs/pipeline/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/n/coxfs01/2p-pipeline/envs/pipeline/lib/python2.7/site-packages/traitlets/config/application

In [2]:
%matplotlib notebook

# Select data

## IGNORE THESE:
##### V1:  JC076-20190501, JC083-20190510
    - if rfs only, ignore: JC083-20190510, JC083-20190511, 
    
    - if blobs & rfs only, ignore: JC083_20190510, JC083-20190511
    
    - if ALL stimuli, ignore: JC083-20190507, JC083-20190510, 
    
#### Lm:  JC078-20190509, JC080-20190506, JC083-20190517
    - if rfs only, ignore: JC078-20190504, JC078-2019513, 
                           JC083-20190512

    - if blobs & rfs: ignore: JC078-20190426, JC078-2019513
                              JC083-20190512
                              
    - if ALL stimuli, ignore: JC078-20190430, JC078-20190426, JC078-20190504
                              JC083-20190508
#### Li:  JC076-20190422, JC099-20190509, JC091-20190607, JC091-20190606
    

In [13]:
always_exclude = ['JC076-20190501', 'JC083-20190510', 
                  'JC078-20190509', 'JC080-20190506', 'JC083-20190517',
                 'JC076-20190422', 'JC099-20190509', 'JC091-20190607', 'JC091-20190606']

also_exclude = ['JC083-20190510', 'JC083-20190511', 
                'JC078-20190504', 'JC078-2019513',
                'JC083-20190512']

excluded_sessions = copy.copy(always_exclude)
excluded_sessions.extend(also_exclude)
print(len(excluded_sessions))

14


# Set responsivity params

In [14]:
responsive_test = None
responsive_thr = None
n_stds = None
experiment_type = 'rfs'

# Get aggregate data

In [15]:
from pipeline.python.classifications import get_dataset_stats as gd
import cPickle as pkl

options = ['-t', 'traces001']
optsE = gd.extract_options(options)

rootdir = optsE.rootdir
aggregate_dir = optsE.aggregate_dir
fov_type = optsE.fov_type
traceid = optsE.traceid
response_type = 'dff'
print aggregate_dir

/n/coxfs01/julianarhee/aggregate-visual-areas


In [99]:
sdata_fpath = os.path.join(aggregate_dir, 'dataset_info.pkl')
if os.path.exists(sdata_fpath):
    with open(sdata_fpath, 'rb') as f:
        sdata = pkl.load(f)
else:
    sdata = gd.aggregate_session_info(traceid=optsE.traceid, trace_type=optsE.trace_type, 
                                       state=optsE.state, fov_type=optsE.fov_type, 
                                       visual_areas=optsE.visual_areas,
                                       blacklist=optsE.blacklist, 
                                       rootdir=optsE.rootdir)
    with open(sdata_fpath, 'wb') as f:
        pkl.dump(sdata, f, protocol=pkl.HIGHEST_PROTOCOL)


In [100]:
stats_desc = util.get_stats_desc(traceid=optsE.traceid,
                                      trace_type= optsE.trace_type,
                                      response_type = response_type,
                                      responsive_test = responsive_test,
                                      responsive_thr = responsive_thr,
                                      n_stds = n_stds)
print(stats_desc)

stats-traces001-corrected-dff-all


In [101]:
dsets = sdata[sdata['experiment'].isin(['rfs', 'rfs10'])] #experiment_type]

# Create output dir for analysis


In [22]:
from pipeline.python.retinotopy import fit_2d_rfs as fitrf

In [26]:
fit_desc = fitrf.get_fit_desc(response_type=response_type)
print fit_desc

fit-2dgaus_dff-no-cutoff


In [29]:
outdir = os.path.join(aggregate_dir, 'receptive-fields', '%s-%s' % (traceid, fit_desc), 'scatter')
if not os.path.exists(outdir):
    os.makedirs(outdir)
print(outdir)

/n/coxfs01/julianarhee/aggregate-visual-areas/receptive-fields/traces001-fit-2dgaus_dff-no-cutoff/scatter


In [19]:
def reformat_morph_values(sdf):
    control_ixs = sdf[sdf['morphlevel']==-1].index.tolist()
    sizevals = np.array([round(s, 1) for s in sdf['size'].unique() if s not in ['None', None] and not np.isnan(s)] )
    sdf.loc[sdf.morphlevel==-1, 'size'] = pd.Series(sizevals, index=control_ixs)
    sdf['size'] = [round(s, 1) for s in sdf['size'].values]

    return sdf


In [20]:
# Set plotting params
threshold_dff = False
min_dff = 0.03
cmap = 'inferno'


In [104]:
def get_rfdata(sdata, fit_desc=None, excluded_sessions=[]):
    assert fit_desc is not None, "No fit-desc specified!"
    
    rfdata = sdata[sdata['experiment'].isin(['rfs', 'rfs10'])]
    for skey in excluded_sessions:
        animalid, session = skey.split('-')
        drop_ixs = rfdata[(rfdata['animalid']==animalid) & (rfdata['session']==session)].index.tolist()
        rfdata= rfdata.drop(drop_ixs)
            
    fit_these = []
    dpaths = {}
    unknown = []
    for (visual_area, animalid, session, fov), g in rfdata.groupby(['visual_area', 'animalid', 'session', 'fov']): #animalid in rfdata['animalid'].unique():
        skey = '-'.join([animalid, session])
        if skey in excluded_sessions:
            continue

        rfruns = g['experiment'].unique()

        for rfname in rfruns:
            if int(session) < 20190511:
                fpath = glob.glob(os.path.join(rootdir, animalid, session, '*%s' % fov, 
                                            'combined_gratings*', 'traces', '%s*' % traceid, 
                                            'receptive_fields', fit_desc, 'fit_results.pkl'))
            else:
                fpath = glob.glob(os.path.join(rootdir, animalid, session, '*%s' % fov, 
                                            'combined_%s_*' % rfname, 'traces', '%s*' % traceid, 
                                            'receptive_fields', fit_desc, 'fit_results.pkl'))
            if len(fpath) > 0:
                assert len(fpath)==1, "TOo many paths: %s" % str(fpath)
                dpaths['-'.join([animalid, session, fov, rfname])] = fpath[0]
                
            elif len(f) == 0:
                fit_these.append((animalid, session, fov, rfname))
            else:
                unknown.append((animalid, session, fov, rfname))
    nostats = []
    for (animalid, session, fov, run_name) in fit_these:
        nostats = resp.visualize_session_stats(animalid, session, fov, altdir=aggregate_session_dir, create_new=True)

    
    print ("N dpaths: %i, N unfit: %i, N unknown: %i" % (len(dpaths), len(fit_these), len(unknown)))
    print("N datasets included: %i, N sessions excluded: %i" % (rfdata.shape[0], len(excluded_sessions)))
    #assert (len(dpaths) + len(fit_these) + len(unknown)) == rfdata.shape[0] - len(excluded_sessions), "There are dsets unaccounted for..."
    
    rfdata = rfdata.reset_index()
    fillpaths = ['' for _ in range(rfdata.shape[0])]
    for skey, fpath in dpaths.items():
        animalid, session, fov, rfname = skey.split('-')
        df_ix = rfdata[ (rfdata['animalid']==animalid) \
                           & (rfdata['session']==session) \
                           & (rfdata['fov']==fov) \
                           & (rfdata['experiment']==rfname)].index.tolist()[0]
        fillpaths[df_ix] = fpath
        
    rfdata['path'] = fillpaths
    
    return rfdata, nostats

In [106]:
rfdata, nostats = get_rfdata(sdata, fit_desc=fit_desc, excluded_sessions=excluded_sessions)

N dpaths: 30, N unfit: 0, N unknown: 0
N datasets included: 30, N sessions excluded: 14


In [108]:
rfdata.head()

,index,animalid,experiment,fov,session,visual_area,exp_no,animal_no,path
0,0,JC084,rfs,FOV1_zoom2p0x,20190522,V1,3,4,/n/coxfs01/2p-data/JC084/20190522/FOV1_zoom2p0...
1,4,JC084,rfs,FOV1_zoom2p0x,20190525,Lm,3,4,/n/coxfs01/2p-data/JC084/20190525/FOV1_zoom2p0...
2,8,JC091,rfs,FOV1_zoom2p0x,20190627,Lm,3,7,/n/coxfs01/2p-data/JC091/20190627/FOV1_zoom2p0...
3,16,JC091,rfs10,FOV1_zoom2p0x,20190614,Li,4,7,/n/coxfs01/2p-data/JC091/20190614/FOV1_zoom2p0...
4,24,JC091,rfs,FOV1_zoom2p0x,20190602,Li,3,7,/n/coxfs01/2p-data/JC091/20190602/FOV1_zoom2p0...


In [111]:
rfdata.iloc[0]['path']

'/n/coxfs01/2p-data/JC084/20190522/FOV1_zoom2p0x/combined_rfs_static/traces/traces001_b78b04_traces001_5c60f6_traces001_7a5ecd_traces001_c80ac8_traces001_656bda_traces001_f17a03/receptive_fields/fit-2dgaus_dff-no-cutoff/fit_results.pkl'

# Select dataset

In [21]:
rootdir = '/n/coxfs01/2p-data'
animalid = 'JC084' #'JC059'
session = '20190522' #'20190227'
fov = 'FOV1_zoom2p0x' #'FOV4_zoom4p0x'
traceid = 'traces001' #'traces001'
response_type = 'dff'


In [30]:
devpath = glob.glob(os.path.join(rootdir, animalid, session, fov, 'combined_*', 
                       'traces', '%s*' % traceid, 'receptive_fields', fit_desc, 'evaluation', 'deviants.json'))[0]
with open(devpath, 'r') as f:
    d = json.load(f)


In [117]:
g['path'].values[0]

'/n/coxfs01/2p-data/JC076/20190502/FOV1_zoom2p0x/combined_gratings_static/traces/traces001_7569de_traces001_48c1d7_traces001_46c882_traces001_676ef4_traces001_c8a2e7/receptive_fields/fit-2dgaus_dff-no-cutoff/fit_results.pkl'

In [119]:
rfdata[rfdata['animalid']=='JC080']

,index,animalid,experiment,fov,session,visual_area,exp_no,animal_no,path
25,95,JC080,rfs,FOV1_zoom2p0x,20190603,Lm,3,2,/n/coxfs01/2p-data/JC080/20190603/FOV1_zoom2p0...
26,103,JC080,rfs10,FOV2_zoom2p0x,20190602,Lm,4,2,/n/coxfs01/2p-data/JC080/20190602/FOV2_zoom2p0...
27,105,JC080,rfs10,FOV1_zoom2p0x,20190602,Li,4,2,/n/coxfs01/2p-data/JC080/20190602/FOV1_zoom2p0...


In [118]:
dlist = []
for (visual_area, animalid, session, fov), g in rfdata.groupby(['visual_area', 'animalid', 'session', 'fov']):
    skey = '-'.join([visual_area, animalid, session, fov])
    print skey
    devpath = os.path.join( os.path.split(g['path'].values[0])[0], 'evaluation', 'deviants.json')
    with open(devpath, 'r') as f:
        d = json.load(f)
    
    dev_list = [('azimuth', i) for i in d['azimuth']]
    dev_list.extend( [('elevation', i) for i in d['elevation']] )
    dev_list.extend([('pass', i) for i in d['pass_rois']])
    tmpd = pd.DataFrame({'condition': [c[0] for c in dev_list],
                         'cell': [c[1] for c in dev_list],
                         'visual_area': [visual_area for _ in range(len(dev_list))],
                         'animalid': [animalid for _ in range(len(dev_list))],
                         'session': [session for _ in range(len(dev_list))],
                         'fov': [fov for _ in range(len(dev_list))]})
    
    dlist.append(tmpd)
    

Li-JC076-20190502-FOV1_zoom2p0x
Li-JC080-20190602-FOV1_zoom2p0x


IOError: [Errno 2] No such file or directory: '/n/coxfs01/2p-data/JC080/20190602/FOV1_zoom2p0x/combined_rfs10_static/traces/traces001_3b2a0a_traces001_72f7e8_traces001_065f6c/receptive_fields/fit-2dgaus_dff-no-cutoff/evaluation/deviants.json'

In [79]:
df = pd.concat(dlist, axis=0)


In [80]:
df.groupby(['visual_area', 'condition']).count()['cell']

visual_area  condition
Li           azimuth       35
             elevation     44
             pass          74
Lm           azimuth      169
             elevation    165
             pass         310
V1           azimuth      162
             elevation    154
             pass         554
Name: cell, dtype: int64

In [ ]:
df[df]

In [61]:
visual_areas = ['V1', 'Lm', 'Li']
colors = ['k', 'royalblue', 'darkorange'] #sns.color_palette(palette='colorblind') #, n_colors=3)
area_colors = {'V1': colors[0], 'Lm': colors[1], 'Li': colors[2]}

In [92]:
ratios = dict((visual_area, {'azimuth': [], 'elevation': []}) for visual_area in visual_areas)
for condition in ['azimuth', 'elevation']:
    for k, g in df.groupby(['visual_area', 'animalid', 'session', 'fov']):
        npass = len(g[g['condition']=='pass'])
        nbad = len(g[g['condition']==condition])
        ratios[k[0]][condition].append(float(nbad)/npass)
        
        

In [93]:
ratios

{'Li': {'azimuth': [1.0, 0.390625], 'elevation': [0.3, 0.640625]},
 'Lm': {'azimuth': [0.3333333333333333,
   0.6851851851851852,
   0.5517241379310345,
   0.2727272727272727,
   0.08333333333333333,
   0.4666666666666667,
   0.42857142857142855,
   0.4473684210526316,
   0.8333333333333334,
   0.4857142857142857,
   0.8333333333333334,
   0.5185185185185185,
   0.5],
  'elevation': [0.3333333333333333,
   0.6296296296296297,
   0.7241379310344828,
   0.36363636363636365,
   0.08333333333333333,
   0.7333333333333333,
   0.5714285714285714,
   0.7105263157894737,
   0.6111111111111112,
   0.37142857142857144,
   0.3333333333333333,
   0.3333333333333333,
   0.375]},
 'V1': {'azimuth': [0.5,
   0.45454545454545453,
   0.40476190476190477,
   0.2962962962962963,
   0.45454545454545453,
   0.35064935064935066,
   0.16853932584269662,
   0.12631578947368421,
   0.18627450980392157,
   0.6041666666666666],
  'elevation': [0.26666666666666666,
   0.18181818181818182,
   0.38095238095238093,


In [97]:
for k, g in df[df['visual_area']=='Li'].groupby(['animalid', 'session']):
    print k

('JC076', '20190502')
('JC091', '20190602')
